# Monitoring setup for Bicycle Sharing Demand Prediction

**Disclaimer**. This example uses the Evidently API as available in version 0.6.7 or lower. Please ensure you are using the correct version when running this notebook. For updated and new examples using the latest Evidently versions, visit our documentation at:  https://docs.evidentlyai.com. 

This notebook shows how you can use the Evidently to:
* calculate performance and data drift for the model, performed as batch checks
* log models quality & data drift using MLflow Tracking
* explore the result

More examples are available in the github: https://github.com/evidentlyai/evidently/tree/main/examples

Evidently docs: https://docs.evidentlyai.com/

Join our Discord: https://discord.com/invite/xZjKRaNp8b

In [2]:
import datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import json

from sklearn import datasets, ensemble, model_selection
from scipy.stats import anderson_ksamp

# from evidently.metrics import RegressionQualityMetric, RegressionErrorPlot, RegressionErrorDistribution
from evidently.metrics import RegressionErrorPlot, RegressionErrorDistribution
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

ImportError: cannot import name 'RegressionErrorPlot' from 'evidently.metrics' (c:\Users\mrcra\Desktop\llm-evaluation-framework\.venv\Lib\site-packages\evidently\metrics\__init__.py)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [ ]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip", verify=False).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'])

In [ ]:
raw_data.index = raw_data.apply(lambda row: datetime.datetime.combine(row.dteday.date(), datetime.time(row.hr)),
                                axis=1)

In [ ]:
raw_data.head()

## Model training

In [ ]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]#'weathersit']

In [ ]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    reference[numerical_features + categorical_features],
    reference[target],
    test_size=0.3
)

In [ ]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
preds_train = regressor.predict(X_train)
preds_test = regressor.predict(X_test)

## Model validation

In [ ]:
X_train['target'] = y_train
X_train['prediction'] = preds_train

X_test['target'] = y_test
X_test['prediction'] = preds_test

In [ ]:
column_mapping = ColumnMapping()

column_mapping.target = 'target'
column_mapping.prediction = 'prediction'
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [ ]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=X_train.sort_index(), current_data=X_test.sort_index(),
                                  column_mapping=column_mapping)
regression_performance_report

## Production model training

In [ ]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

In [ ]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [ ]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
reference['prediction'] = ref_prediction

In [ ]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=None, current_data=reference,
                                  column_mapping=column_mapping)
regression_performance_report

## Some time has passed - how is the model working?

In [ ]:
current_prediction = regressor.predict(current[numerical_features + categorical_features])
current['prediction'] = current_prediction

### Week 1

In [ ]:
regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
                                  column_mapping=column_mapping)
regression_performance_report

### Week 2

In [ ]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'],
                                            column_mapping=column_mapping)
regression_performance_report.show()

### Week 3

In [ ]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'],
                                            column_mapping=column_mapping)
regression_performance_report.show()

## Why the quality has dropped?

In [ ]:
column_mapping_drift = ColumnMapping()

column_mapping_drift.target = target
column_mapping_drift.prediction = prediction
column_mapping_drift.numerical_features = numerical_features
column_mapping_drift.categorical_features = []

In [ ]:
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])

data_drift_report.run(
    reference_data=reference,
    current_data=current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'],
    column_mapping=column_mapping_drift,
)
data_drift_report

## Let's customize the report!

In [ ]:
from evidently.core import ColumnType
from evidently.calculations.stattests import StatTest, register_stattest

def _anderson_stat_test(reference_data: pd.Series, current_data: pd.Series, feature_type: ColumnType, threshold: float):
    p_value = anderson_ksamp([reference_data, current_data])[2]
    return p_value, p_value < threshold

anderson_stat_test = StatTest(
    name="anderson",
    display_name="Anderson test (p_value)",
    allowed_feature_types=[ColumnType.Numerical]
)

register_stattest(anderson_stat_test, default_impl=_anderson_stat_test)
# options = DataDriftOptions(feature_stattest_func=anderson_stat_test, all_features_threshold=0.9, nbinsx=20)

In [ ]:
the_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution(),
    DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
])


the_report.run(
    reference_data=reference,
    current_data=current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'],
    column_mapping=column_mapping_drift
)
the_report

## Jupyter vs Automated Run?

To run this part of the tutorial you need to install MLflow (if not installed yet)

You can install MLflow with the following command: `pip install mlflow` or install MLflow with scikit-learn via `pip install mlflow[extras]`

More details:https://mlflow.org/docs/latest/tutorials-and-examples/tutorial.html#id5

In [ ]:
try:
    import mlflow
except:
    !pip install mlflow

In [ ]:
import mlflow
#import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [ ]:
experiment_batches = [
    ('2011-01-29 00:00:00','2011-02-07 23:00:00'),
    ('2011-02-07 00:00:00','2011-02-14 23:00:00'),
    ('2011-02-15 00:00:00','2011-02-21 23:00:00'),
]

In [ ]:
the_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution(),
    DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
])

the_report.run(
    reference_data=reference,
    current_data=current.loc[experiment_batches[0][0]:experiment_batches[0][1]],
    column_mapping=column_mapping_drift
)

In [ ]:
logged_json = json.loads(the_report.json())

In [ ]:
logged_json

In [ ]:
[x['metric'] for x in logged_json['metrics']]

In [ ]:
logged_json['metrics'][0]['result']['current']['mean_error']

In [ ]:
logged_json['metrics'][3]['result']['drift_share']

In [ ]:
#log into MLflow
client = MlflowClient()

#set experiment
mlflow.set_experiment('Model Quality Evaluation')

#start new run
for date in experiment_batches:
    with mlflow.start_run() as run: #inside brackets run_name='test'

        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Get metrics
        the_report = Report(metrics=[
            RegressionQualityMetric(),
            RegressionErrorPlot(),
            RegressionErrorDistribution(),
            DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
        ])
        the_report.run(
            reference_data=reference,
            current_data=current.loc[date[0]:date[1]],
            column_mapping=column_mapping_drift)
        logged_json = json.loads(the_report.json())

        me = logged_json['metrics'][0]['result']['current']['mean_error']
        mae = logged_json['metrics'][0]['result']['current']["mean_abs_error"]
        drift_share = logged_json['metrics'][3]['result']['drift_share']

        # Log metrics
        mlflow.log_metric('me', round(me, 3))
        mlflow.log_metric('mae', round(mae, 3))
        mlflow.log_metric('drift_share', round(drift_share, 3))

        print(run.info)

In [ ]:
#run MLflow UI (NOT recommended! It will be more convinient to run it directly from the TERMINAL)
#!mlflow ui

# Support Evidently
Enjoyed the tutorial? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently